# Scientific Computing
## Week 22 - Sparse linear algebra
## Matthew Hennessy

# Overview

* Introduce the idea of sparse matrices 
* Showcase SciPy's `sparse` module for sparse linear algebra

# Sparse matrices

* A **sparse** matrix is a matrix that only has a few non-zero entries in each row
* The identity matrix is a perfect example of a sparse matrix
* A **dense** matrix is a matrix that has mostly non-zero entries

* Sparse matrices commonly appear when solving PDEs numerically
* The tridiagonal $\mathbf{A}^{DD}$ matrix that appears when using the finite difference method is sparse 
\begin{align*}
\mathbf{A}^{DD} = 
\begin{pmatrix}
-2 & 1 &  &  &  &  \\
1 & -2 & 1 & & &  \\
 & 1 & -2 & 1 &  &  \\
 &  & \ddots& \ddots& \ddots & \\
 &  & & 1 & -2 & 1 \\
 &  & &  & 1 & -2
\end{pmatrix}
\end{align*}

# Memory considerations

* Dense matrices are stored as two-dimensional arrays (such as NumPy arrays)
* Every entry in the matrix is stored in memory
* Sparse matrices can be stored using data types that reduce memory consumption
* The idea is to only store the values and indices of non-zero entries in a sparse matrix

# Example

Consider the diagonal matrix
\begin{align}
A = \begin{pmatrix} 1.0 & 0 & 0\\ 0 & 2.0 & 0 \\ 0 & 0 & 3.0 \end{pmatrix}
\end{align}

Treating this as a dense matrix requires storing 9 floats (= $9 \times 8$ bytes = $72$ bytes)

Treating this as a sparse matrix requires storing three arrays (values, row indices, column indices)
```python
values = [1.0, 2.0, 3.0]
rows = [0, 1, 2]
cols = [0, 1, 2]
```

This requires storing three floats and six integers: $3 \times 8$ bytes + $6 \times 4$ bytes = $48$ bytes)

# Sparse storage formats

* There are many different ways to store sparse matrices. 
* Coordinate (COO) format: This is similar to the example on the previous slides, where the values and the coordinates (rows, cols) are stored
* Compressed Sparse Column (CSC) format: Similar to COO but duplicate entries in the column array are removed
* [Wikipedia](https://en.wikipedia.org/wiki/Sparse_matrix) has a good summary of the different formats and theirs pros/cons

# Sparse matrices in SciPy

* SciPy's `sparse` module contains functions for creating sparse matrices
* `scipy.sparse.eye(N)` creates a sparse $N \times N$ identity matrix
* `scipy.sparse.diags` creates a sparse diagonal matrix
* The sparse storage format can be specified using the `format` argument

# Example

* Let's explore the memory savings of using sparse matrices for the identity matrix
* We'll first create the matrix using NumPy and calculate how much memory it takes up
* Then we'll repeat the process using SciPy's sparse module

Using NumPy:

In [28]:
import numpy as np

N = 5000
I_dense = np.eye(N)
size_in_bytes = I_dense.nbytes
size_in_MB = size_in_bytes / 1e6
print(size_in_MB, 'MB required to store as a dense NumPy array')

200.0 MB required to store as a dense NumPy array


Using SciPy and storing in COO format

In [29]:
import scipy.sparse as sp

N = 5000
I_sp = sp.eye(N, format = 'coo')
size_in_bytes = I_sp.data.nbytes + I_sp.row.nbytes + I_sp.col.nbytes
size_in_MB = size_in_bytes / 1e6
print(size_in_MB, 'MB required to store as a sparse COO array')

0.08 MB required to store as a sparse COO array


# Speed

* Aside from being memory efficient, sparse matrices can greatly accelerate matrix operations

In [26]:
b = np.random.random(N)

# compute I * b where I is a dense identity matrix
%timeit I_dense @ b

12.6 ms ± 432 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
# compute I * b where I is a sparse identity matrix
%timeit I_sp @ b

14 µs ± 230 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Sparse linear algebra

* SciPy can perform operations from linear algebra on sparse matrices
* Need to use functions in the `scipy.sparse.linalg` module
* `spsolve(A, b)` solves the linear system $Ax = b$ when $A$ is a sparse matrix
* **Do not** use NumPy functions with sparse matrices

# Example

Solving $Ax = b$ where $A$ is a sparse diagonal matrix with random entries

First, we'll do this using NumPy

In [32]:
import numpy as np

N = 5000
d = np.random.random(N)
b = np.random.random(N)

A_dense = np.diag(d)

%timeit np.linalg.solve(A_dense, b)

753 ms ± 42.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now, using SciPy

In [34]:
import scipy.sparse.linalg as sla

A_sparse = sp.diags(d, format = 'csc')

%timeit sla.spsolve(A_sparse, b)

1.77 ms ± 51.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Using sparse matrices is more than 400 times faster!

# Summary

* Sparse matrices contain only a few non-zero entries per row
* Sparse matrices can be stored efficiently, reducing memory usage and speeding up operations
* SciPy contains functions for creating sparse matrices and operating on them